In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import cmocean
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import seaborn as sns

In [ ]:
ystart, yend = "2020", "2039"

# load 1x1 degree data as downloaded from the lra
ds = xr.open_mfdataset("/scratch/m/m218027/2t*.nc", concat_dim="time", combine="nested").sortby("time")
ds = ds.isel(time=np.unique(ds.time.values, return_index=True)[1]).sel(time=slice(ystart, yend))

popt = ds.polyfit(dim="time", deg=1)
ns_in_month = np.timedelta64(1, "M").astype("timedelta64[ns]").astype(int)

In [ ]:
fig, ax = plt.subplots(
    figsize=(10, 5),
    constrained_layout=True,
    subplot_kw={"projection": ccrs.EqualEarth(0*-135)},
)
ax.set_global()

# Use high-resolution coastlines (10m resolution for better detail)
coastline = cf.NaturalEarthFeature(
    category="physical", name="coastline", scale="50m", edgecolor="black", facecolor="none"
)
ax.add_feature(coastline, linewidth=0.5)  # Adjust linewidth for visibility
#ax.coastlines(lw=0.7)
#ax.add_feature(cf.BORDERS, lw=0.4)
ax.add_feature(cf.RIVERS, lw=0.4, edgecolor="black")

# Reduce the thickness of the map outline (bounding box)
for spine in ax.spines.values():
    spine.set_linewidth(0.5)
    
sm = ax.pcolormesh(
    ds.lon,
    ds.lat,
    popt["2t_polyfit_coefficients"].sel(degree=1) * ns_in_month * 10,
    transform=ccrs.PlateCarree(),
    vmin=-0.5,
    vmax=0.5,
    cmap="cmo.balance",
)
ax.set_title(f"Linear trend in 2m air temperature ({ystart}–{yend}) in ICON (SSP3-7.0)")
cbar=fig.colorbar(sm, label="K / decade", pad=0.1, shrink=0.5,aspect=30,extend="both",orientation="horizontal")
# Remove the bounding box around the colorbar
cbar.outline.set_visible(False)
sns.set_context('paper')
fig.savefig(f"climatedt_icon_2t_trend_{ystart}-{yend}.png", dpi=250)

fig, ax = plt.subplots(figsize=(10, 5))
ds["2t"].mean(("lat", "lon")).plot()
sns.set_context('paper')
sns.despine(offset=10)
fig.savefig(f"climatedt_icon_2t_timeseries.png", dpi=250)